### 결측치 없는 X, Y 컬럼 추가하기.

- 각각 원본 csv 파일에 열 방향으로 concat 하면 됨.
- 예 : dt = pd.concat([dt1, dt2], axis=1)

In [113]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

In [114]:
isTrainSelected = 0

if isTrainSelected:
    origin_filepath = 'data/train.csv'
    target_filepath = 'data/newXY_for_train.csv'
else:
    origin_filepath = 'data/test.csv'
    target_filepath = 'data/newXY_for_test.csv'

In [115]:
dt = pd.read_csv(origin_filepath)

In [116]:
# Train data와 Test data shape은 아래와 같습니다.
print('Train data shape : ', dt.shape)

Train data shape :  (9272, 51)


In [117]:
dt[dt['번지'].isnull()]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
4963,서울특별시 서초구 내곡동,NaN,0.0,0.0,서초포레스타2단지,84.48,202308,19,12,2015,...,위탁,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0
4964,서울특별시 서초구 내곡동,NaN,0.0,0.0,서초포레스타2단지,84.48,202308,22,11,2015,...,위탁,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0


In [118]:
dt.loc[dt['아파트명'] == '서초포레스타2단지', '번지'] = '384'
dt[dt['아파트명'] == '서초포레스타2단지']

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
4963,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.48,202308,19,12,2015,...,위탁,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0
4964,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.48,202308,22,11,2015,...,위탁,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0


In [119]:
dt[dt['번지'].isnull()]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일


In [120]:
dt.loc[dt['아파트명'] == '힐스테이트 서초 젠트리스', '번지'] = '557'
dt[dt['아파트명'] == '힐스테이트 서초 젠트리스']

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일


In [121]:
dt[dt['번지'].isnull()]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일


In [122]:
dt['지번주소'] = (dt['시군구'] + " " + dt['번지']).astype(str)
dt

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,지번주소
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.9700,202307,26,5,1987,...,4858.00,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,서울특별시 강남구 개포동 658-1
1,서울특별시 강남구 개포동,651-1,651.0,1.0,개포더샵트리에,108.2017,202308,15,10,2021,...,2724.46,305.0,의무,2022-02-23 13:01:10.0,Y,N,127.056394,37.484892,2022-02-23 11:05:05.0,서울특별시 강남구 개포동 651-1
2,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,161.0000,202307,28,15,1984,...,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,서울특별시 강남구 개포동 652
3,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,133.4600,202308,10,14,1984,...,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,서울특별시 강남구 개포동 652
4,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,104.4300,202308,18,6,1984,...,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,서울특별시 강남구 개포동 652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9267,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.6500,202307,19,13,2014,...,14171.00,1568.0,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 816
9268,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.6200,202307,25,12,2014,...,14171.00,1568.0,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 816
9269,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,101.6500,202308,27,12,2014,...,14171.00,1568.0,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 816
9270,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.9400,202309,2,18,2014,...,14171.00,1568.0,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 816


In [123]:
dt_jibun_xy = pd.read_csv('data/jibun_xy_by_naver.csv')
dt_jibun_xy.drop(columns='Unnamed: 0', inplace=True)
dt_jibun_xy

,지번주소,좌표X,좌표Y
0,서울특별시 강남구 개포동 1164-12,127.052949,37.473193
1,서울특별시 강남구 개포동 1164-13,127.053094,37.473287
2,서울특별시 강남구 개포동 1164-14,127.053239,37.473382
3,서울특별시 강남구 개포동 1164-20,127.054110,37.473948
4,서울특별시 강남구 개포동 1164-25,127.054827,37.474392
...,...,...,...
8948,서울특별시 중랑구 중화동 438,127.077761,37.604575
8949,서울특별시 중랑구 중화동 450,127.081923,37.597155
8950,서울특별시 중랑구 중화동 452,127.078533,37.599890
8951,서울특별시 중랑구 중화동 453,127.080084,37.602687


In [124]:
dt = pd.merge(dt, dt_jibun_xy, on='지번주소', how='left')
dt

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X_x,좌표Y_x,단지신청일,지번주소,좌표X_y,좌표Y_y
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.9700,202307,26,5,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,서울특별시 강남구 개포동 658-1,127.056859,37.476276
1,서울특별시 강남구 개포동,651-1,651.0,1.0,개포더샵트리에,108.2017,202308,15,10,2021,...,의무,2022-02-23 13:01:10.0,Y,N,127.056394,37.484892,2022-02-23 11:05:05.0,서울특별시 강남구 개포동 651-1,127.057257,37.484829
2,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,161.0000,202307,28,15,1984,...,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,서울특별시 강남구 개포동 652,127.056019,37.483975
3,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,133.4600,202308,10,14,1984,...,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,서울특별시 강남구 개포동 652,127.056019,37.483975
4,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,104.4300,202308,18,6,1984,...,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,서울특별시 강남구 개포동 652,127.056019,37.483975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9267,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.6500,202307,19,13,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 816,127.106467,37.617195
9268,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.6200,202307,25,12,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 816,127.106467,37.617195
9269,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,101.6500,202308,27,12,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 816,127.106467,37.617195
9270,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.9400,202309,2,18,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 816,127.106467,37.617195


In [125]:
display(dt['좌표X_x'].isnull().sum())
display(dt['좌표Y_x'].isnull().sum())

display(dt['좌표X_y'].isnull().sum())
display(dt['좌표Y_y'].isnull().sum())

6562

6562

2

2

In [126]:
dt[dt['좌표X_y'].isnull()].head(20)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X_x,좌표Y_x,단지신청일,지번주소,좌표X_y,좌표Y_y
4963,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.48,202308,19,12,2015,...,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,서울특별시 서초구 내곡동 384,NaN,NaN
4964,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.48,202308,22,11,2015,...,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,서울특별시 서초구 내곡동 384,NaN,NaN


In [127]:
dt.loc[dt['아파트명'] == '서초포레스타2단지', '좌표X_y'] = 127.0609502
dt.loc[dt['아파트명'] == '서초포레스타2단지', '좌표Y_y'] = 37.4555506
dt[dt['아파트명'] == '서초포레스타2단지']

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X_x,좌표Y_x,단지신청일,지번주소,좌표X_y,좌표Y_y
4963,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.48,202308,19,12,2015,...,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,서울특별시 서초구 내곡동 384,127.06095,37.455551
4964,서울특별시 서초구 내곡동,384,0.0,0.0,서초포레스타2단지,84.48,202308,22,11,2015,...,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0,서울특별시 서초구 내곡동 384,127.06095,37.455551


In [128]:
dt[dt['좌표X_y'].isnull()].head(20)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X_x,좌표Y_x,단지신청일,지번주소,좌표X_y,좌표Y_y


In [129]:
dt.loc[dt['아파트명'] == '힐스테이트 서초 젠트리스', '좌표X_y'] = 127.0576544
dt.loc[dt['아파트명'] == '힐스테이트 서초 젠트리스', '좌표Y_y'] = 37.4562853
dt[dt['아파트명'] == '힐스테이트 서초 젠트리스']

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X_x,좌표Y_x,단지신청일,지번주소,좌표X_y,좌표Y_y


In [130]:
display(dt['번지'].isnull().sum())
display(dt['지번주소'].isnull().sum())
display(dt['좌표X_y'].isnull().sum())
display(dt['좌표Y_y'].isnull().sum())

0

0

0

0

In [131]:
dt.rename(columns={'좌표X_x':'좌표X', '좌표Y_x':'좌표Y', '좌표X_y':'좌표X_2', '좌표Y_y':'좌표Y_2'}, inplace=True)
dt

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,지번주소,좌표X_2,좌표Y_2
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.9700,202307,26,5,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,서울특별시 강남구 개포동 658-1,127.056859,37.476276
1,서울특별시 강남구 개포동,651-1,651.0,1.0,개포더샵트리에,108.2017,202308,15,10,2021,...,의무,2022-02-23 13:01:10.0,Y,N,127.056394,37.484892,2022-02-23 11:05:05.0,서울특별시 강남구 개포동 651-1,127.057257,37.484829
2,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,161.0000,202307,28,15,1984,...,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,서울특별시 강남구 개포동 652,127.056019,37.483975
3,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,133.4600,202308,10,14,1984,...,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,서울특별시 강남구 개포동 652,127.056019,37.483975
4,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,104.4300,202308,18,6,1984,...,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,서울특별시 강남구 개포동 652,127.056019,37.483975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9267,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.6500,202307,19,13,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 816,127.106467,37.617195
9268,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.6200,202307,25,12,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 816,127.106467,37.617195
9269,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,101.6500,202308,27,12,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 816,127.106467,37.617195
9270,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.9400,202309,2,18,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 816,127.106467,37.617195


In [132]:
dt.drop(columns='지번주소', inplace=True)
dt

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,좌표X_2,좌표Y_2
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.9700,202307,26,5,1987,...,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,127.056859,37.476276
1,서울특별시 강남구 개포동,651-1,651.0,1.0,개포더샵트리에,108.2017,202308,15,10,2021,...,305.0,의무,2022-02-23 13:01:10.0,Y,N,127.056394,37.484892,2022-02-23 11:05:05.0,127.057257,37.484829
2,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,161.0000,202307,28,15,1984,...,419.0,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,127.056019,37.483975
3,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,133.4600,202308,10,14,1984,...,419.0,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,127.056019,37.483975
4,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,104.4300,202308,18,6,1984,...,419.0,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,127.056019,37.483975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9267,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.6500,202307,19,13,2014,...,1568.0,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,127.106467,37.617195
9268,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.6200,202307,25,12,2014,...,1568.0,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,127.106467,37.617195
9269,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,101.6500,202308,27,12,2014,...,1568.0,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,127.106467,37.617195
9270,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.9400,202309,2,18,2014,...,1568.0,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,127.106467,37.617195


In [133]:
dt[['좌표X_2', '좌표Y_2']]

,좌표X_2,좌표Y_2
0,127.056859,37.476276
1,127.057257,37.484829
2,127.056019,37.483975
3,127.056019,37.483975
4,127.056019,37.483975
...,...,...
9267,127.106467,37.617195
9268,127.106467,37.617195
9269,127.106467,37.617195
9270,127.106467,37.617195


In [134]:
dt[['좌표X_2', '좌표Y_2']].to_csv(target_filepath, encoding='utf-8-sig', index=False)